# Get Betas for Comps

### Necessary Imports

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

### S&P 500 Data

In [2]:
# download S&P 500 data
sp500 = yf.download('^GSPC', start='2020-10-17', end='2022-10-17', interval='1wk')
sp500 = sp500.dropna()
sp500 = sp500.reset_index()

# get the weekly returns
sp500['weekly_return'] = sp500['Adj Close'].pct_change()
sp500 = sp500.dropna()


[*********************100%***********************]  1 of 1 completed


# Calculate the betas 

In [3]:
def beta(ticker, start, end, interval):
    # get data
    data = yf.download(ticker, start=start, end=end, interval=interval)
    data = data.dropna()
    data = data.reset_index()
    
    # calculate weekly returns
    data['weekly_return'] = data['Adj Close'].pct_change()
    
    # drop na
    data = data.dropna()
    
    # merge with sp500 data on date
    data = data.merge(sp500, on='Date', suffixes=('', '_sp500'))

    # run a regression
    X = data['weekly_return_sp500']
    y = data['weekly_return']

    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    
    return model.params[1]

### Uber Beta

In [4]:
print('UBER Beta:', beta('UBER', '2020-10-17', '2022-10-17', '1wk'))

[*********************100%***********************]  1 of 1 completed
UBER Beta: 1.6069961556587666


### Stocks in each Segment

In [5]:
mobility = ['LYFT', 'DIDIY', 'GRAB', 'BRDS', 'HLBZ', 'NIU']
delivery = ['DASH', 'DROOF', 'DLVHF', 'HLFFF', 'APRN']
freight = ['CHRW', 'XPO', 'DPSGY', 'GXO', 'JBHT', 'HUBG', 'LSTR']

### Mobility Betas

In [6]:
mobility_beta = pd.DataFrame()

for ticker in mobility:
    mobility_beta[ticker] = [beta(ticker, '2020-10-17', '2022-10-17', '1wk')]

mobility_beta = mobility_beta.T
mobility_beta.columns = ['mobility_beta']

mobility_beta.loc['Average Beta'] = mobility_beta.mean()


mobility_beta

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,mobility_beta
LYFT,1.757576
DIDIY,2.458666
GRAB,1.293262
BRDS,0.775504
HLBZ,1.420304
NIU,1.430427
Average Beta,1.522623


### Delivery Betas

In [7]:
delivery_beta = pd.DataFrame()

for ticker in delivery:
    delivery_beta[ticker] = [beta(ticker, '2020-10-17', '2022-10-17', '1wk')]

delivery_beta = delivery_beta.T
delivery_beta.columns = ['Delivery_Beta']

delivery_beta.loc['Average Beta'] = delivery_beta.mean()

delivery_beta

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Delivery_Beta
DASH,1.732550
DROOF,0.570273
DLVHF,1.411151
HLFFF,0.813275
APRN,1.152158
Average Beta,1.135881


### Freight Betas

In [8]:
freight_beta = pd.DataFrame()

for ticker in freight:
    freight_beta[ticker] = [beta(ticker, '2020-10-17', '2022-10-17', '1wk')]

freight_beta = freight_beta.T
freight_beta.columns = ['Freight_Beta']

freight_beta.loc['Average Beta'] = freight_beta.mean()

freight_beta

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Freight_Beta
CHRW,0.803364
XPO,1.405871
DPSGY,1.003579
GXO,1.635069
JBHT,0.978679
HUBG,0.944203
LSTR,0.721046
Average Beta,1.070259


### Average Betas per Segment

In [9]:
all_betas = mobility_beta.merge(delivery_beta, left_index=True, right_index=True)
all_betas = all_betas.merge(freight_beta, left_index=True, right_index=True)

all_betas = all_betas.rename(index={'Average Beta': 'Beta'})
all_betas['UBER'] = beta('UBER', '2020-10-17', '2022-10-17', '1wk')

all_betas

[*********************100%***********************]  1 of 1 completed


,mobility_beta,Delivery_Beta,Freight_Beta,UBER
Beta,1.522623,1.135881,1.070259,1.606996


# Cash Adjusted Betas